### LIBRERIAS NECESARIAS PARA LAS FUNCIONES

In [1]:
import pandas as pd
import numpy as np

### ARCHIVOS NECESARIOS PARA LAS FUNCIONES

In [13]:
# ARCHIVOS NECESARIOS COMPLETOS. De todas formas en cada función se levantan solo las columnas necesarias para el desarrollo de cada una particularmente

# df_steam = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', on_bad_lines='skip')
# df_items = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\items_post_etl.csv", sep=';', on_bad_lines='skip')
# df_reviews = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\reviews_post_etl.csv", sep=';', on_bad_lines='skip')

###  FUNCIONES

##### N° 1 def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. 

In [21]:
def developer( desarrollador : str ):
    """Cantidad de items y porcentaje de contenido Free 
        por año según empresa desarrolladora"""
     
    df_steam = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', on_bad_lines='skip', usecols=['developer', 'year', 'price']) 
        
    #Filtrar juegos por desarrollador
    df_desarrollador = df_steam[df_steam['developer'] == desarrollador].copy()
    
    # Contar la cantidad de items por año
    cantidad_items_por_year = df_desarrollador.groupby('year').size().reset_index(name="cantidad_items")
    
    # Contar la cantidad de items GRATUITOS por año
    cantidad_items_gratuitos_por_year = df_desarrollador[df_desarrollador["price"] == 0].groupby('year').size().reset_index(name='cantidad_items_gratuitos_por_año').astype('Int64')

    # Combinar filtros para dar resultado y creación de columna que calcule el porcentaje sobre el total
    resultado = pd.merge(cantidad_items_por_year, cantidad_items_gratuitos_por_year, on='year', how='left').fillna(0)
    resultado['% Contenido Free'] = ((resultado['cantidad_items_gratuitos_por_año'] / resultado['cantidad_items']) * 100).round(2)
    
    # Convertir el DataFrame a un diccionario
    resultado_dict = resultado.to_dict(orient='records')
    
    return resultado_dict

developer("SmiteWorks USA, LLC") #probando función


[{'year': 2014,
  'cantidad_items': 74,
  'cantidad_items_gratuitos_por_año': 0,
  '% Contenido Free': 0.0},
 {'year': 2015,
  'cantidad_items': 80,
  'cantidad_items_gratuitos_por_año': 0,
  '% Contenido Free': 0.0},
 {'year': 2016,
  'cantidad_items': 231,
  'cantidad_items_gratuitos_por_año': 0,
  '% Contenido Free': 0.0},
 {'year': 2017,
  'cantidad_items': 428,
  'cantidad_items_gratuitos_por_año': 4,
  '% Contenido Free': 0.93}]

#### N° 2 def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items. 


In [7]:
# Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

def userdata( User_id : str ):
    """Devuelve cantidad de dinero gastado por el usuario, el porcentaje de 
    recomendación en base a reviews.recommend y cantidad de items"""

    df_items = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\items_post_etl.csv", sep=';', on_bad_lines='skip',usecols=['user_id', 'item_id', 'price'])
    df_reviews = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\reviews_post_etl.csv", sep=';', on_bad_lines='skip',usecols=['user_id','recommend'])
    
    #Filtrar por user_id
    df_user = df_items[df_items['user_id']== User_id]

    #Calcular el gasto total por  usuario
    gasto = str(round(df_user['price'].sum(),2))+' USD'

    #Calcular la cantidad de items comprados
    cantidad_items = df_user['item_id'].count()
    
    # Filtrar reviews por ususario 
    df_reviews_usuario = df_reviews[df_reviews['user_id']== User_id]
    
    #Calcular la cantidad de juegos recomendados (True)
    cantidad_recomendaciones = (df_reviews_usuario['recommend'] == True).sum()

    # Calcular la cantidad de juegos que tienen datos en la columna review
    cantidad_juegos_con_review = df_reviews_usuario['recommend'].notna().sum()

    #Porcentaje de recomendación
    if cantidad_items > 0:
        porcentaje_recomendaciones = str(round(cantidad_recomendaciones * 100 / cantidad_juegos_con_review, 2)) + '%'
    else:
        porcentaje_recomendaciones = '0%'

    #Muestra los resultados en un diccionario y con el formato adecuado
    return {"Id Usuario" : str(User_id), 
           "Dinero gastado": str(gasto), 
           "% de recomendación": str(porcentaje_recomendaciones),
           "Cantidad de items": int(cantidad_items)  
           }
userdata("STEAM0082987612")

{'Id Usuario': 'STEAM0082987612',
 'Dinero gastado': '1209.26 USD',
 '% de recomendación': '85.71%',
 'Cantidad de items': 106}

#### N° 3 def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.


In [16]:
# Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

def UserForGenre( genero : str ): 
    """Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista 
    de la acumulación de horas jugadas por año de lanzamiento."""
    
    df_items = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\items_post_etl.csv", sep=';', on_bad_lines='skip', usecols=['user_id', 'item_id', 'playtime_forever']) 
    df_steam = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', on_bad_lines='skip', usecols=['item_id', 'year',"genres"])
    
    genero = genero.lower()
    genero = genero.title()

    # Filtrar el DataFrame df_steam para obtener los juegos del género especificado
    df_steam_filtrado  = df_steam[df_steam['genres'].apply(lambda x: genero in x)]

    # Unir df_steam_filtrado con df_items basado en 'item_id' para obtener los usuarios y horas jugadas para los juegos del género
    df_combinado = pd.merge(df_steam_filtrado[['item_id', 'year',"genres"]], df_items[['user_id', 'item_id', 'playtime_forever']], on='item_id', how='inner')

    # Agrupar por 'user_id' y sumar las horas jugadas para cada usuario
    horas_por_usuario = df_combinado.groupby('user_id')['playtime_forever'].sum().reset_index()

    # Obtener el usuario con más horas jugadas
    usuario_mas_horas = horas_por_usuario.loc[horas_por_usuario['playtime_forever'].idxmax(), 'user_id']

    # Agrupar por año y sumar las horas jugadas para cada año
    horas_por_año_usuario = df_combinado[df_combinado['user_id'] == usuario_mas_horas].groupby('year')['playtime_forever'].sum().reset_index()

    # Convertir a la lista deseada de acumulación de horas jugadas por año
    dicc_horas_por_year = [{'Año': row['year'], 'Horas': row['playtime_forever']} for index, row in horas_por_año_usuario.iterrows()]

    respuesta = {
    f"Usuario con más horas jugadas para Género {genero}": usuario_mas_horas,
    "Horas jugadas": dicc_horas_por_year }

    return respuesta

UserForGenre("Sports") 

{'Usuario con más horas jugadas para Género Sports': '76561197995725436',
 'Horas jugadas': [{'Año': 2012.0, 'Horas': 40941.0},
  {'Año': 2013.0, 'Horas': 79186.0},
  {'Año': 2014.0, 'Horas': 73079.0}]}

#### N° 4 def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)


In [11]:
# Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}
def best_developer_year( year : int ):
    """Devuelve el top 3 de desarrolladores con juegos 
    MÁS recomendados por usuarios para el año dado. """
    
    df_steam = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', on_bad_lines='skip', usecols=['item_id', 'year',"developer"])
    df_reviews = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\reviews_post_etl.csv", sep=';', on_bad_lines='skip',usecols=['user_id','item_id','recommend'])
    
    # Filtrar df_steam por el año dado
    juegos_por_año = df_steam[df_steam['year'] == year]

    # Unir df_steam con df_reviews por item_id
    merged_df = pd.merge(juegos_por_año, df_reviews, on='item_id', how='left')

    # Filtrar solo las recomendaciones positivas
    recomendaciones_positivas = merged_df[merged_df['recommend'] == True]

    # Contar recomendaciones por desarrollador
    conteo_recomendaciones = recomendaciones_positivas.groupby('developer').size().reset_index(name='cantidad_recomendaciones')

    # Ordenar df para obtener los primeros 3 desarrolladores segun la cantidad de recomendaciones
    top_desarrolladores = conteo_recomendaciones.sort_values(by='cantidad_recomendaciones', ascending=False).head(3)

    # Guardo los tres primeros puestos
    primer_puesto = top_desarrolladores.iloc[0]['developer']
    segundo_puesto = top_desarrolladores.iloc[1]['developer']
    tercer_puesto = top_desarrolladores.iloc[2]['developer']

    # Formatear el resultado
    resultado = {
        "Puesto 1": primer_puesto,
        "Puesto 2": segundo_puesto,
        "Puesto 3": tercer_puesto
    }
    return resultado

best_developer_year(2017)

{'Puesto 1': 'Smartly Dressed Games',
 'Puesto 2': 'Freejam',
 'Puesto 3': 'Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC'}

#### N° 5 def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.


In [12]:
# Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}
def developer_reviews_analysis( desarrolladora : str ): 
    """Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y 
    una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados 
    con un análisis de sentimiento como valor positivo o negativo."""
    
    df_steam = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', on_bad_lines='skip', usecols=['item_id',"developer"])
    df_reviews = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\reviews_post_etl.csv", sep=';', on_bad_lines='skip',usecols=['user_id','item_id','sentiment_analysis'])
    
    
    # Filtrar df_steam por la desarrolladora especificada
    juegos_desarrolladora = df_steam[df_steam['developer'] == desarrolladora]
    
    # Unir df_steam con df_reviews por item_id
    merged_df = pd.merge(juegos_desarrolladora, df_reviews, on='item_id', how='left')

    # Contar la cantidad total de reseñas positivas y negativas
    conteo_reseñas = merged_df['sentiment_analysis'].value_counts().to_dict()

        # Crear el diccionario de resultados
    reseñas = {
        desarrolladora: [
            conteo_reseñas.get(2.0, 0),   # Positive count
            conteo_reseñas.get(0.0, 0)      # Negative count
        ]
    }

    # Renombrar claves en la lista a 'Positivas' y 'Negativas'
    reseñas[desarrolladora] = {
        "Positivas": reseñas[desarrolladora][0],
        "Negativas": reseñas[desarrolladora][1]
    }

    return reseñas

developer_reviews_analysis("Team Psykskallar")

{'Team Psykskallar': {'Positivas': 33, 'Negativas': 34}}